In [1]:
using Random
using DelimitedFiles
using PrettyTables
using Distributions
using Plots
using LaTeXStrings

In [2]:
function demand_generator_mat(N_dem, demand_length, dM, demand_type, std_dev)
    d_mat = zeros(demand_length, N_dem)
    for j in 1:N_dem
        #This segment generates uniform demands, of length demand_length,in the interval [-d_max,d_max] 
        if demand_type == "uniform"
            d_mat[:,j] = Int.(round.(rand(Uniform(0,dM),demand_length)))  
        end
        ##This segment generates approximately normally distributed demands
        if demand_type == "normal"
            d_mat[:,j] = Int.(round.(dM .+ rand(Normal(0, std_dev), demand_length)))
        end
        ##This segment generates demands with a Poisson distribution
        if demand_type == "poisson"
            d_mat[:,j] = Int.(round.(dM .+ rand(Poisson(0, std_dev), demand_length)))
         end
    end
    return d_mat           
end


function printTable(data, header)
    println("Results")
    pretty_table(data; header=header,formatters=ft_printf("%5.3f",1:11))
end


function plotData(evolution, ylabel, title)
    gr()
    plot(evolution, layout = 4, seriestype=:scatter, label=false,xlabel=[L"k" L"k" L"k" L"k"],ylabel=ylabel, title=title,
        palette=cgrad.([:grays :blues :heat :lightrainbow]), bg_inside=[:lightblue :lightblue :lightblue :lightblue])
end

plotData (generic function with 1 method)

In [3]:
# Parameters
horiz = 20           # total horizon
n_ser = 2            # number of servers
YM = 10              # max buffer legnth before dropping calls
XM = 6               # max queue length 
phiM = 4             # max adimission to queue

t = 1:horiz

d_fn = "..//CC_simple//d_mat_Thu_22_May_2025_19_39_50.txt";
a_fn = "..//CC_simple//a_mat_Thu_22_May_2025_19_39_50.txt";
d_mat = DelimitedFiles.readdlm(d_fn);
a_mat = DelimitedFiles.readdlm(a_fn);

# d_mat = demand_generator_mat(1, 20, 5, "uniform", 1)
# a_mat = demand_generator_mat(1,20,1,"uniform",0.5)

d = d_mat[1:horiz, 1];  # demand for incoming calls
a = a_mat[1:horiz, 1];  # abandonment for calls

In [4]:
# State variables
X = zeros(Int, horiz+1)     # current number of customers in queue x(k)
s = zeros(Int, horiz+1)     # number of served customers per timestep
dr = zeros(Int, horiz)      # dropped due to full buffer
z = zeros(Int, horiz+1)     # custumers served

q = zeros(Int, horiz+1)     # custumers entering queue
n = zeros(Int, horiz+1)     # number of empty slots in the queue

X[1] = 3  # initial state
s[1] = n_ser
z[1] = 0

for k in 1:horiz
    n[k] = XM - X[k] +a[k] + s[k]
    q[k] = min(n[k], d[k])

    X[k+1] = X[k] + q[k] - a[k] - s[k]

    # If queue exceeds max length, drop excess
    if d[k] > n[k]
        dr[k] = d[k] - n[k]  # dropped calls due to full queue
    else
        dr[k] = 0
    end
    
    z[k+1] = z[k] + s[k]
    # Serve up to number of servers
    s[k+1] = min(X[k], n_ser)
end

In [5]:
data = hcat(t, X[1:horiz,1], s[1:horiz,1], dr, d_mat[:,1], a_mat[:,1])
header = (["t", "X", "s", "dr", "d_mat", "a_mat"])

printTable(data, header)

Results
┌────────┬───────┬───────┬────────┬────────┬───────┐
│      t │     X │     s │     dr │  d_mat │ a_mat │
├────────┼───────┼───────┼────────┼────────┼───────┤
│  1.000 │ 3.000 │ 2.000 │  2.000 │  7.000 │ 0.000 │
│  2.000 │ 6.000 │ 2.000 │  0.000 │  3.000 │ 1.000 │
│  3.000 │ 6.000 │ 2.000 │  0.000 │  3.000 │ 2.000 │
│  4.000 │ 5.000 │ 2.000 │  3.000 │  7.000 │ 1.000 │
│  5.000 │ 6.000 │ 2.000 │  9.000 │ 12.000 │ 1.000 │
│  6.000 │ 6.000 │ 2.000 │  5.000 │  9.000 │ 2.000 │
│  7.000 │ 6.000 │ 2.000 │  3.000 │  6.000 │ 1.000 │
│  8.000 │ 6.000 │ 2.000 │  0.000 │  3.000 │ 1.000 │
│  9.000 │ 6.000 │ 2.000 │  0.000 │  0.000 │ 1.000 │
│ 10.000 │ 3.000 │ 2.000 │  3.000 │  9.000 │ 1.000 │
│ 11.000 │ 6.000 │ 2.000 │  8.000 │ 11.000 │ 1.000 │
│ 12.000 │ 6.000 │ 2.000 │  9.000 │ 12.000 │ 1.000 │
│ 13.000 │ 6.000 │ 2.000 │  8.000 │ 12.000 │ 2.000 │
│ 14.000 │ 6.000 │ 2.000 │  8.000 │ 12.000 │ 2.000 │
│ 15.000 │ 6.000 │ 2.000 │  0.000 │  2.000 │ 0.000 │
│ 16.000 │ 6.000 │ 2.000 │  0.000 │  0

In [12]:
# State variables
X = zeros(Int, horiz+1)     # current number of customers in queue x(k)
s = zeros(Int, horiz)       # number of active servers
dr = zeros(Int, horiz)      # dropped due to full buffer
z = zeros(Int, horiz+1)     # custumers served

q = zeros(Int, horiz+1)     # custumers entering queue
n = zeros(Int, horiz+1)     # number of empty slots in the queue

# Servers
t_ser = [2, 3] # delay to serve one customer on
t_serM = 1 + maximum(t_ser) # max time to serve one customer

sl = zeros(Int, horiz+1)                    # number of empty servers
s_st  = zeros(Int, horiz+1, n_ser)          # servers state, where 0 means free and > 1 means busy
se = zeros(Int, horiz+1, n_ser, t_serM)     # server conveyors
s_in = zeros(Int, horiz+1, n_ser, t_serM)   # matrix of customers entering the servers

c_in = zeros(Int, horiz+1)    # customers entering the servers
c_out = zeros(Int, horiz+1)   # number of customers leaving the servers

X[1] = 3  # initial state
s[1] = 0
z[1] = 0



for k in 1:horiz

    X[k] = XM - X[k] + a[k] + c_in[k]
    q[k] = min(n[k], d[k])
    s[k] = n_ser # all servers active 

    s_st[k, :] = sum(se[k, :, :], dims=2) 
    sl[k] = s[k] - sum(s_st[k, :])  # number of free servers

    c_in[k+1] = min(X[k], sl[k])

    s_aux = c_in[k] 
    s_in[k,:,:] = zeros(Int, n_ser, t_serM) 
    for i in 1:n_ser
        if s_st[k,i] == 0 && s_aux > 0  
            s_aux -= 1
            s_in[k, i, t_serM-t_ser[i]] = 1
            # s_in[k, i, rand(1:t_serM)] = 1  # assign a customer to random empty slot in convveyor
        end
    end        

    se[k,:,:] += s_in[k,:,:]
    se[k+1, :, 2:t_serM] = se[k, :, 1:t_serM-1]  # shift the state of server i
    se[k+1, :, 1] = zeros(n_ser, 1) # reset the first element of conveyor

    c_out[k+1] = sum(se[k,:,t_serM])  # number of customers leaving the servers

    # println(se[k, :, :], s_st[k,:], s_in[k,:,:])
    X[k+1] = X[k] + q[k] - c_in[k] -a[k]

    # If queue exceeds max length, drop excess
    if d[k] > n[k]
        dr[k] = d[k] - n[k]  # dropped calls due to full queue
    else
        dr[k] = 0
    end
        
    z[k+1] = z[k] + c_out[k]
end

In [13]:
data = hcat(t, X[1:horiz,1], c_in[1:horiz,1], c_out[1:horiz, 1], s_st[1:horiz, 1], s_st[1:horiz, 2],  dr, z[1:horiz,1], d_mat[:,1], a_mat[:,1])
header = (["t", "X", "c_in", "c_out", "st_ser1", "st_ser2", "dr", "z", "d_mat", "a_mat"]);
printTable(data, header)

Results
┌────────┬───────┬───────┬───────┬─────────┬─────────┬────────┬───────┬────────┬───────┐
│      t │     X │  c_in │ c_out │ st_ser1 │ st_ser2 │     dr │     z │  d_mat │ a_mat │
├────────┼───────┼───────┼───────┼─────────┼─────────┼────────┼───────┼────────┼───────┤
│  1.000 │ 3.000 │ 0.000 │ 0.000 │   0.000 │   0.000 │  7.000 │ 0.000 │  7.000 │ 0.000 │
│  2.000 │ 6.000 │ 2.000 │ 0.000 │   0.000 │   0.000 │  3.000 │ 0.000 │  3.000 │ 1.000 │
│  3.000 │ 7.000 │ 2.000 │ 0.000 │   1.000 │   1.000 │  3.000 │ 0.000 │  3.000 │ 2.000 │
│  4.000 │ 4.000 │ 0.000 │ 0.000 │   1.000 │   1.000 │  7.000 │ 0.000 │  7.000 │ 1.000 │
│  5.000 │ 4.000 │ 0.000 │ 1.000 │   0.000 │   1.000 │ 12.000 │ 0.000 │ 12.000 │ 1.000 │
│  6.000 │ 6.000 │ 1.000 │ 1.000 │   0.000 │   0.000 │  9.000 │ 1.000 │  9.000 │ 2.000 │
│  7.000 │ 6.000 │ 2.000 │ 0.000 │   1.000 │   0.000 │  6.000 │ 2.000 │  6.000 │ 1.000 │
│  8.000 │ 5.000 │ 1.000 │ 0.000 │   1.000 │   1.000 │  3.000 │ 2.000 │  3.000 │ 1.000 │
│  9.000 │ 4.

In [ ]:
assignments[:, t_serM] .= free_servers_mask .* (customers_to_serve .> 0)

In [ ]:
free_servers_mask = s_st[5, :] .== 0
println(free_servers_mask)
customers_to_serve = min(c_in[5], sum(free_servers_mask))
println(2)

assignments = zeros(Int, n_ser, t_serM)  # matrix of assignments

assignments[:, t_serM] .= free_servers_mask .* (customers_to_serve .> 0)


Bool[0, 1]
1


2-element view(::Matrix{Int64}, :, 4) with eltype Int64:
 0
 1

In [12]:
n_ser = 5

customers_to_serve = 2
# Create a vector for the condition
condition_vector = fill(customers_to_serve > 0, n_ser)


5-element Vector{Bool}:
 1
 1
 1
 1
 1